# <center> <h1>Data description - exercices </h1> </center>

<img src="../images/ab_testing.png" width="200">

_TODO :
    schema echantillonnage
    ajout de la variable N dans le notebook exercices_

# Pourquoi faire un A/B testing ?

L'A/B testing permet de tester une différence de résultat en fonction d'un paramètre à deux modalités. Cette technique est notamment utile pour estimer si un nouveau produit ou une nouvelle feature à un impact positif sur le résultat attendu (taux de clics, de conversions, etc.) en comparant 2 groupes d’utilisateurs que l’on expose à des features/produits différent.e.s.

### Quelques cas d'usage classics

L'AB testing permer de valider la valeur ajoutée : 
* d’un nouvel algorithme de recommandation
* d’un changement de backend
* d’une modification de wording sur un site internet 
* d’un changement de disposition des éléments ou des couleurs d'une page

### Des cas peu propices et des solutions alternatives

Lorsque l'effet d'une modification est complexe ou à long terme, l'A/B testing est moins facile à mettre en place. Par exemple lors d'un changement de logo, les clients peuvent mettre du temps à s'habituer au changement. Dans ce cas l'A/B testing ne montrera pas forcémment les effet bénéfiques à long terme.

Il existe des alternatives à l'A/B testing dans ces cas : 
* l'analyse des logs
* la mise en place de focus groups et/ou d'enquêtes pour obtenir des réponses à des questions plus complexes
* une évaluation humaine par des experts métier

Dans certains cas également l'A/B testing est trop coûteux pour être mis en place. Dans le domaine médical, par exemple, il peut être très coûteux (financièrememt mais surtout moralement et humainement) d'appliquer le moins bon traitement à tout un groupe (A ou B). 

Ce problème est complexe car on ne connaît pas le meilleur traitement à l'avance, mais il est parfois possible de réaliser un A/B testing en formulant le use-case comme un problème de _multi-armed bandit_ et de le résoudre avec des techniques de reinforcement learning simple mais efficace pour minimiser le __regret__.

<center> 
    <img src="../images/warning.png" width="200">
</center>

<center> 
    <span; style="margin-top:200px ; color:red;font-size:20px">
    Comparer juste des moyennes ne permet pas de conclure ! Même si le résultat semble évident.
    </span>
</center>

In [ ]:
get_ipython().magic(u'matplotlib inline')
%run -i ../utils/credentials.py
%run -i ../utils/imports.py
%run -i ../utils/plots.py
%run -i ../utils/stats.py

# Rappel

Une variable aléatoire représente le résultat d'une expérience qui n'est pas entièrement déterminée à l'avance. 

Cette variable est souvent représentée par une loi déterminant la manière dont vont se répartir les résultats d'expériences identiques répétées un grand nombre de fois.

Considérons le cas suivant : 
* le phénomène : l'erreur lors d'une mesure
* le nombre d'expériences : 100 000

La loi du phénomène peut dans une première approche être représentée par les effectifs pour chaque interval de valeur. Le graphique suivant montre trois cas différents.

In [ ]:
# nombre de tirages
nb_sample = 100000

# écarts-types pour les tirages
std1 = 5
std2 = 3
moy = 4

# tirages
x1 = np.random.normal(0,std1,nb_sample)
x2 = np.random.normal(0,std2,nb_sample)
x3 = np.random.normal(moy,std2,nb_sample)

layout = go.Layout(xaxis=dict(title="valeurs"), 
                   yaxis=dict(title="effectifs"),
                   width=800,height=600)

data = [go.Histogram(x=x1, name='écart-type = {}'.format(std1)), 
        go.Histogram(x=x2, name='écart-type = {}'.format(std2)),
        go.Histogram(x=x3, name='moyenne = {}'.format(moy))]

fig = go.Figure(data, layout)

In [ ]:
iplot(fig, filename='')

En théorie on caractérise une loi grâce à sa fonction de densité. 

Pour une loi normale deux paramètres suffisent pour calculer la fonction de densité : sa moyenne ($\mu$) et son écart-type ($\sigma$). Cette loi est notée $\mathcal{N}(\mu,\sigma)$.

ATTENTION : certains auteurs utilisent la moyenne et la variance ($\sigma^2$) pour décrire la même chose : $\mathcal{N}(\mu,\sigma^2)$

In [ ]:
loi = scs.norm(0,3)

x1 = np.linspace(-10, 10, 1000)
y1 = loi.pdf(x1)

data = [go.Scatter(x=x1,y=y1)]
layout = go.Layout(width=800,height=500)

fig = go.Figure(data,layout)

Graphiquement la fonction de densité de la loi est représenté par une courbe ayant l'allure de celle ci-dessous.

In [ ]:
iplot(fig, filename='')

La probabilité d'avoir une valeur entre a et b est égale à l'air sous la courbe entre ces deux valeurs.

In [ ]:
loi = scs.norm(0,1)

x1 = np.linspace(-4, 4, 1000)
y1 = loi.pdf(x1)

x2 = np.linspace(-1.96, 1.96, 1000)
y2 = loi.pdf(x2)

trace1 = go.Scatter(x=x1,y=y1, name="densité")
trace2 = go.Scatter(x=x2,y=y2,fill='tozeroy',mode= 'none',name="proba")

data = [trace1,trace2]
layout = go.Layout(width=800,height=500)

fig = go.Figure(data,layout)

In [ ]:
iplot(fig, filename='')

# Le principe du test
On constitue un groupe A (de taille $N_A$) à qui on applique un paramètre et un groupe B (de taille $N_B$) à qui on applique l'autre paramètre.

Warning : il faut choisir les individus au hasard au sein de chaque groupe sinon on ne teste pas les paramètres mais les caractéristiques de sélection des groupes.

### Plus précisément

On cherche à répondre à la question suivante : les moyennes des deux séries $(x^A_1,...,x^A_{N_A})$ et $(x^B_1,...,x^B_{N_B})$ sont elles égales ou différentes ?

$$\bar{x}^A = \bar{x}^B \space ?$$


**Les hypothèses du test**
- $H_0$ : $$\bar{x}^A = \bar{x}^B \space$$


- $H_1$ : $$\bar{x}^A > \bar{x}^B \space$$

Pour le moment on fait l'hypothèse que N est relativement grand ($N_A$>30 et $N_B$>30).

__Idée centrale du test : sous l'hypothèse nulle l'écart entre les deux moyennes suit une loi normale centrée en 0 et de variance connue.__

**Si H0 est vraie**

La différence entre les deux moyennes normalisée (avec les écarts-types) devrait être proche de 0. 

Plus précisémment, le théorème central limite nous permet de dire que la valeur cette différence peut être approximée par une loi normale centrée en 0. 

Cela signifie que si l'on calcule la moyenne sur deux échantillons (ceux correspondant à nos groupes A et B) et que l'on répète cette opération plusieurs fois, la différence entre les deux moyennes sera rarement très éloignée de 0. 

Formellement cela s'écrit : $$\bar{X}^A - \bar{X}^B\rightarrow \mathcal{N}\Big(0,\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}\Big)$$

Avec : $$ S_J^2 = \frac{1}{(N_J-1)}\sum_i (x_i - \bar{x_i})^2$$ 

### Risque de première espèce : rejeter l'hypothèse nulle à tort

Dans un cas concret on ne peut pas répéter plusieur fois l'opération de calcul de moyenne sur plusieurs couples d'échantillons A et B. On a une seule valeur qui est le résultat de notre test et que l'on appelle $t_{AB}$.

Pour décider si on accepte l'hypothèse nulle on va fixer une valeur et si $t_{AB}$ est supérieur à cette valeur on va rejeter l'hypothèse d'égalité des moyennes.

Pour cela on fixe l'intervalle de valeurs pour $t_{AB}$ dans lequel on accèpte notre hypothèse. Par exemple on accèpte notre hypothèse nulle si notre valeur $t_{AB}$ est comprise dans la zone où se situent 95% des valeurs théoriques.

In [ ]:
loi = scs.norm(0,1)
p = 0.95
plot = plot_ppf(p=p,loi=loi, threshold = loi.ppf(p))

### Risque de deuxième espèce : accépter l'hypothèse nulle à tort

Le risque de deuxième espèce correspond au risque d'accépter l'hypothèse nulle dans le cas où l'écart réel entre les moyennes est à un niveau donné non nul.

On parle de __puissance du test__ pour désigner la valeur complémentaire de ce risque. Par exemple lorsque le risque de second espèce est de 20%, la puissance du test est de 80%.

Lorsque l'on considère que l'écart réel entre les deux moyennes est très faible, on fait face à un risque de seconde espèce élevé. Par exemple si l'écart entre les deux moyennes est de 0.5 avec un écart-type estimé de 1, le risque de deuxième espèces est de 87%.

In [ ]:
loi1 = scs.norm(0,1)
loi2 = scs.norm(0.5,1)
t = loi1.ppf(0.95)
plot_cdf_2_lois(loi1, loi2, threshold=False)(t)

Plus l'écart entre les deux moyennes est élevé, plus le risque de seconde espèce est faible.

In [ ]:
loi1 = scs.norm(0,1)
loi2 = scs.norm(3,1)
t = loi1.ppf(0.95)
plot_cdf_2_lois(loi1, loi2)(t)

De la même manière si les variances empiriques sont faibles (les mesures sont plus précises), le risque de seconde espèce est réduit. La séparation est "plus nette".

In [ ]:
loi1 = scs.norm(0,0.2)
loi2 = scs.norm(0.5,0.2)
t = loi1.ppf(0.95)
plot_cdf_2_lois(loi1, loi2)(t)

# En pratique comment ça se passe ?

## A/ On commence par estimer la taille de l'échantillon nécessaire

En pratique on calcule la taille de l'échantillon qui a "_de bonnes chances_" de nous permettre de détecter l'effet minimal qui nous intéresse avec des niveaux de risque réduits. Pour cela il faut faire des hypothèses sur la variance des résultats $\sigma^2$.

Les propriétés du test permettent de calculer la taille de l'échantillon permettant de respecter les niveaux de risque souhaités.

$$N_A^{min} = N_B^{min} = \frac{2\times\sigma^2\times(Z_{\alpha} + Z_{\beta})^2}{(mde)^2} $$

Avec :
* $Z_\alpha$ : la valeur au delà de laquelle on rejette $H_0$ avec un risque de 1er espèce $\alpha$
* $Z_\beta$ : la valeur (modulo le $mde$) en deça de laquelle on accepte $H_0$ avec un risque de 2ème espèce $\beta$
* $\sigma^2$ : la variance hypothétique des résultats
* $mde$ : l'effet minimum détectable souhaité

In [ ]:
def size_sample_AB_test(risk1, risk2, var, mde, bilateral = False):
    
    if bilateral :
        Z_alpha = scs.norm(0,1).ppf(1 - risk1/2)
    else :    
        Z_alpha = scs.norm(0,1).ppf(1 - risk1)
    
    Z_beta = scs.norm(0,1).ppf(1 - risk2)

    min_N = (2*(var)*(Z_beta + Z_alpha)**2/ mde**2)
    
    return min_N


Par exemple pour tester une différence de taux de clic, avec : 
* un risque de première espèce de 5%
* un risque de deuxième espèce de 20%
* un taux de clics de base de 5%
* une différence détectable minimale de 1.5 points

In [ ]:
risk1 = 0.05
risk2 = 0.2
p = 0.5
mde = 0.05
pooled_prob = (2*p + mde) / 2
var = pooled_prob*(1-pooled_prob) # ici on estime la variance grâce à la formule de la variance d'une loi de Bernouilli 
                                  # ceci se justifie par le fait que l'événement (clic ou non) est booléen (1 ou 0).

In [ ]:
sample_size = size_sample_AB_test(risk1 = risk1, risk2 = risk2, var = var, mde = mde)
sample_size = int(sample_size)
sample_size

## B/ Une fois les données récupérées on procède au test

On utilise la formule vue plus haut :
    $$\bar{X}^A - \bar{X}^B\rightarrow \mathcal{N}\Big(0,\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}\Big)$$
    
Qui est équivalent à la formule suivante :

$$\frac{\bar{X}^A - \bar{X}^B}{\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}}\rightarrow \mathcal{N}\Big(0, 1\big)$$

Notre statistique de test sera donc $t_{AB} = \frac{\bar{X}^A - \bar{X}^B}{\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}} $ que l'on va comparer au seuil fixé pour respecter le risque de première espèce.

In [ ]:
def test_H0(x_A, x_B, risk1, bilateral=False):
    
    # Calcul de t_AB : différence des moyennes normalisée
    diff_mean = (np.mean(x_B) - np.mean(x_A)) # diff entre les moyennes
    std_pooled = np.sqrt((np.var(x_A)/len(x_A)) + (np.var(x_A)/len(x_A))) # estimation de l'écart-type joint
    stat = diff_mean/std_pooled # différence normalisée
    
    # Seuil correspondant au risk de première espèce défini
    if bilateral :
        t = scs.norm(0,1).ppf(1 - risk1/2)
    else :
        t = scs.norm(0,1).ppf(1 - risk1)

    return (stat < t), t

On vérifie sur une simulation qu'avec la taille d'échantillon choisi, le risque de deuxième espèce sera proche de celui ciblé.

In [ ]:
results_test_simu = []
mean_A = p
mean_B = p + mde

for ii in range(0,10000):
    
    x_A = np.random.binomial(1, mean_A, size=sample_size)
    x_B = np.random.binomial(1, mean_B, size=sample_size)
    
    # vaut 1 si H_0 est acceptée (à tort ici car mean_A <> mean_B)
    results_test_simu.append(test_H0(x_A, x_B, risk1)[0])

"Calcul par simulation du risque de deuxième espèce : {}%".format(int((np.mean(results_test_simu))*100))

In [ ]:
results_test_simu = []
mean_A = p
mean_B = p

for ii in range(0,10000):
    
    x_A = np.random.binomial(1, mean_A, size=sample_size)
    x_B = np.random.binomial(1, mean_B, size=sample_size)
    
    # vaut 1 si H_0 est accepté (à raison ici car mean_A = mean_B)
    results_test_simu.append(test_H0(x_A, x_B, risk1)[0])

"Calcul par simulation de l'erreur de première espèce du test : {}%".format(int((1 - np.mean(results_test_simu))*100))

## C/ Si on rejette l'hypothèse nulle, on calcule l'intervalle de confiance du résultat

In [ ]:
x_A = np.random.binomial(1, mean_A, size=sample_size)
x_B = np.random.binomial(1, mean_A + mde, size=sample_size)
test_H0(x_A, x_B, risk1)

In [ ]:
def confidence_interval_diff(x_A, x_B, sig_level=0.05):
    mean_diff = np.mean(x_B) - np.mean(x_A)
    var_diff = np.var(x_B)/len(x_B) + np.var(x_A)/len(x_A)
    
    gap = (scs.norm(0,1).ppf(1 - sig_level/2))*np.sqrt(var_diff)
    
    min_ = mean_diff - gap
    max_ = mean_diff + gap
    
    return(min_,max_)

In [ ]:
confidence_interval_diff(x_A, x_B) # x_B - x_A

## D/ Si on accepte l'hypothèse nulle, on précise la puissance du test ou le risque de deuxième espèce

On fait l'hypothèse que l'écart vaut le minimum détectable effect. On a alors : 

$$\frac{\bar{X}^A - \bar{X}^B}{\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}}\rightarrow \mathcal{N}\Big(\frac{mde}{\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}}, 1\Big)$$

In [ ]:
N_A = 2900
N_B = 2900

x_A = np.random.binomial(1, p, size=N_A)
x_B = np.random.binomial(1, p + mde, size=N_B)

var_A = np.var(x_A)
var_B = np.var(x_B)

In [ ]:
def power_test(var_A, var_B ,mde ,risk1) :
    std_pooled = np.sqrt(var_A/N_A + var_B/N_B)
    expectation = mde/std_pooled
    t = test_H0(x_A, x_B, risk1)[1]
    
    return (1 - scs.norm(expectation,1).cdf(t))

In [ ]:
power_test(var_A,var_B,mde,risk1)

# Pour aller plus loin

### Des groupes de taille différentes

Si on souhaite des taille d'échantillon différentes on va utiliser $r$ la proportion entre les deux pour calculer la taille d'échantillon :

$$N_{A} = \frac{r+1}{r} \times \frac{\sigma^2\times(Z_{\alpha} + Z_{\beta})^2}{(mde)^2} $$


$$N_{B} = r \times N_{A}$$

### Des effectifs réduits

Si on souhaite un test pour des tailles d'échantillon plus petits on utilise une loi de student plutôt qu'une loi normale. On utilise alors la formule suivante

$$\frac{\bar{X}^A - \bar{X}^B}{\sqrt{\frac{S_A^2}{N_A} + \frac{S_B^2}{N_B}}} \rightarrow \mathcal{student}\big(N_A + N_B -2 \big)$$

Dans ce cas on fait deux hypothèses suplémentaires : 
* les variances sont normales ;
* les variances sont proches(facteur 3)

### Le problème du _multi-armed bandit_

Lorsque le coût d'un A/B testing classic est trop élevé, il devient nécessaire de minimiser le regret, c'est-à-dire l'écart à l'optimal absolu.

Une solution est de reformuler le problème d'A/B testing comme un problème de _multi-armed bandit_. On a plusieurs choix possibles (par exemple "A", "B" et "C") rapportant chaucun un _"revenu"_ moyen différent mais inconnu a priori. 

La question est alors de savoir quelle option choisir pour un échantillon de n individus ?

La difficulté va être d'arbitrer entre explorer les choix tout en réduisant le regret.

Formellement le regret est : $$ r_n = n\times\mu^* - \sum_{k=1}^{n}\mathbb{E}[\mu_{I_k}]$$

Avec : 
* n : la taille de l'échantillon
* $\mu^*$ : la moyenne pour le meilleur choix
* $I_k$ : le choix pour l'individu k (ici $I_k \in \{A,B,C\}$)
* $\mathbb{E}[\mu_{I_k}]$ : le revenu espéré pour le choix $I_k$

### Une solution au problème : l'algorithme UCB (upper confidence bound)

La stratégie peut être résumée comme suit : on choisit l'option qui rapporte le plus en moyenne avec un bonus d'autant plus grand que l'option a été peu choisie. L'idée est de choisir l'option qui, lorsque l'on est optimiste, a le plus de chance de rapporter (lorsque l'on est optimiste une option peu explorée est supposée rapporter plus). On a alors deux cas de figure :
* soit l'option rapporte beaucoup est alors on maintien le regret à un niveau bas
* soit l'option rapporte peu et on apprend beaucoup (le résultat est très éloigné de l'attente)

Formellement, à chaque étape k (pour chaque individu k) on calcule la valeur de l'UCB pour chacune des options $I\in \{A,B,C\}$ : 
$$ UCB_I(k) = \bar{\mu}^{k-1}_{I} + \sqrt{\frac{2ln(k)}{T_I(k-1)}} $$

Avec :
* $\bar{\mu}^{k-1}_{I}$ : la moyenne empirique des revenus générés par l'option I après k-1 essais
* $T_I(k-1)$ : le nombre de fois que l'option I a été choisie avec k-1 essais

On choisit alors l'option dont l'UCB est la plus élevée : 
$$ I_k = argmax_{I} UCB_I(k)$$

### Quand utiliser ce type d'algorithmes ?
L'algorithme UCB est réalisé par étape, ce qui nécessite d'attendre le résultat pour un individu avant the faire le choix de l'option pour l'individu suivant. Lorsque le résultat du choix n'est récupérable que longtemps après, il n'est pas envisageable de reformuler le problème comme un problème de multi-armed bandit (par exemple, pour tester les effets à long terme d'un médicament).

__Le fait d'utiliser un algorithme de RL n'est _a priori_ pas incompatible avec les étapes classiques de l'A/B testing :__

* estimer la taille de l'échantillon nécessaire (bien qu'il est en théorie possible d'appliquer l'algorithme de reinforcement learning sur tous les individus à venir - à long terme c'est, dans les cas simples, la meilleure option qui sera choisie systématiquement)
* réaliser un test de significativité des différences (si on souhaite choisir l'une des options de manière définitive)
* donner un intervalle de confiance autour des moyennes (cela peut être important pour évaluer le ROI notamment)

<center> <h1>Take away</h1> </center>

<img src="./images/coffee.png" width="200">


__Expresso__ : 

* Comparer deux moyennes _"à l'oeil nu"_ ne suffit pas
* Il faut préparer son test en lien avec le métier pour définir les risques acceptables et l'effet minimum détectable recherché
* Il faut constituer ses échantillons de manière aléatoire
* Si une différence n'est pas significative, cela ne veut pas dire qu'il n'y a pas de différence du tout ! Cela peut venir d'un manque de données et donc de précision
* Lorsqu'on accepte l'hypothèse nulle, on indique la puissance du test pour le mde défini
* Lorsque l'on rejette l'hypothèse nulle, on analyse un intervalle de confiance plutôt qu'une valeur unique

__Sugar Story__ :

* Des AB testing sont parfois réalisés _"à l'oeil nu"_ (même dans des grands groupes)
* Savoir calculer des intervalles de confiance de tête permet parfois d'éviter des boulettes (oups !)

# Références : 
* Tests paramétriques de comparaison de 2 moyennes, José LABARERE, Université Joseph Fourier de Grenoble
* https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f

# Get more on my github <img src="./images/github.png" width="100">
https://github.com/JJublanc/statistics_tools

In [ ]:
# jupyter nbconvert --to slides AB_testing.ipynb --post serve